<a href="https://colab.research.google.com/github/Jesse-abu/ML-projects/blob/main/Copy_of_fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_data = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
test_data = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])

In [ ]:
train_message = train_data["message"].values.tolist()
train_label = np.array([0 if x=="ham" else 1 for x in train_data['label'].values.tolist()])
test_message = test_data["message"].values.tolist()
test_label = np.array([0 if x=="ham" else 1 for x in test_data['label'].values.tolist()])

In [ ]:
vocabulary_dict = {}
for massage in train_message:
  for vocabulary in massage.split():
    if vocabulary not in vocabulary_dict:
      vocabulary_dict[vocabulary] = 1
    else:
      vocabulary_dict[vocabulary] += 1

In [ ]:
vocab = len(vocabulary_dict)
max_len = len(max(train_message, key=len).split())

In [ ]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Embedding, Dense
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
encoded_train_message = [one_hot(d, vocab) for d in train_message]
padded_train_message = pad_sequences(encoded_train_message, maxlen=max_len, padding='post')
encoded_test_message = [one_hot(d, vocab) for d in test_message]
padded_test_message = pad_sequences(encoded_test_message, maxlen=max_len, padding='post')

In [ ]:
model = keras.Sequential()

embedding_layer = keras.layers.Embedding(vocab, 100, input_length=max_len)
model.add(embedding_layer)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(1, activation='sigmoid'))

monitor = EarlyStopping(monitor='val_acc', min_delta=1e-4, patience=25, verbose=1, mode='max', restore_best_weights=True)



In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
history = model.fit(padded_train_message, train_label, validation_data=(padded_test_message, test_label), callbacks=[monitor], epochs=10, verbose=2)

In [ ]:
#pred = model.evaluate(test_data)
text = "how would you like to win $9000"
encoded = one_hot(text, vocab)
padded = pad_sequences([encoded], maxlen=max_len, padding='post')
pred = model.predict(padded)
print(pred[0][0])

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  text = pred_text
  encoded = one_hot(text, vocab)
  padded = pad_sequences([encoded], maxlen=max_len, padding='post')
  pred = model.predict(padded)
  sham = 'spam' if pred > 0.01 else 'ham'
  prediction = [pred[0][0], sham]
  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
